<a href="https://colab.research.google.com/github/Devirajana/-Team-10-travel-Itinerary-planner/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [32]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [33]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [34]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [35]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [36]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [37]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [38]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [39]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [40]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [41]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [42]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [43]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [44]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [45]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [46]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, **averaging the validation accuracy across multiple splits** generally **gives more consistent results**, and this is a core benefit of techniques like **cross-validation**. Here's why:

### 1. **Reduces Variance in Results**
   - **Single split**: If you evaluate your model using a single training/validation split, the performance can be influenced by the specific data points in that split. If the split happens to be unrepresentative (e.g., it’s not a good representation of the overall population or is highly skewed), the accuracy can be misleading.
   - **Multiple splits**: When you perform **cross-validation** (such as **k-fold cross-validation**), you train and evaluate the model multiple times on different splits of the data. By averaging the validation accuracy across multiple splits, you reduce the influence of any single unrepresentative split. This provides a more reliable estimate of the model's generalization ability across various subsets of the data.

### 2. **Improves Model Evaluation**
   - Averaging across multiple splits helps smooth out fluctuations in model performance due to randomness in how data is split. For example:
     - If you randomly partition the data and evaluate on just one split, you may encounter overfitting (if the validation set is too similar to the training set) or underfitting (if the validation set is very different).
     - By using multiple splits, you ensure that the model is evaluated in a more robust and varied context, which leads to a more comprehensive assessment of its performance.

### 3. **More Reliable Estimate of Generalization Performance**
   - Cross-validation gives a better sense of how the model will perform on **unseen data**. It provides a more accurate estimate of how well the model will generalize when deployed to a real-world dataset, especially if you are working with limited data. By averaging across multiple validation sets, you're effectively testing the model on various subsets of the data, making the result more generalizable.

### 4. **Helps with Hyperparameter Tuning**
   - In scenarios where you are fine-tuning hyperparameters (e.g., in grid search or random search), using multiple splits can help determine the best configuration with more confidence. Relying on a single split might lead to over-optimizing the hyperparameters for a specific subset of the data, which may not generalize well to other data splits.

### 5. **Cross-Validation Methods**
   - **k-fold cross-validation**: The most common approach, where the data is divided into **k** parts (or folds). The model is trained on **k-1** folds and validated on the remaining fold. This process is repeated **k** times, each time with a different fold as the validation set. The average validation accuracy across all **k** runs gives a more stable estimate of the model's performance.
   - **Stratified k-fold cross-validation**: For classification problems, particularly when classes are imbalanced, stratified k-fold ensures that each fold has a similar distribution of classes. This ensures that the model is tested on representative subsets.

### 6. **Averaging in Practice**
   - If you were to run a **5-fold cross-validation** and get validation accuracies of:
     - Fold 1: 82%
     - Fold 2: 78%
     - Fold 3: 85%
     - Fold 4: 81%
     - Fold 5: 79%
     
     The average validation accuracy would be:
     \[
     \text{Average} = \frac{82 + 78 + 85 + 81 + 79}{5} = 81%
     \]
     This average result is much more reliable than a single accuracy score from just one fold or one validation split, as it incorporates the model’s performance across multiple data subsets.

### 7. **Dealing with Model Instability**
   - Some models can be unstable and may vary significantly in performance due to minor changes in the data (for example, neural networks, which can be sensitive to random initialization). Averaging across multiple splits helps mitigate the risk of reporting an overly optimistic or pessimistic accuracy score based on one particular split.

### Conclusion
Averaging the validation accuracy across multiple splits, especially via **cross-validation**, leads to **more consistent** and **reliable results**. It reduces the risk of bias due to random variability in data splitting, smooths out fluctuations in performance, and provides a better estimate of how the model will perform in real-world, unseen scenarios. Therefore, cross-validation is widely regarded as a standard practice for evaluating model performance.


2. Does it give more accurate estimate of test accuracy?

**Averaging validation accuracy across multiple splits** (like in **cross-validation**) **does not directly give you an estimate of test accuracy**, but it does **provide a more reliable estimate** of how well your model will perform on unseen data, which can be thought of as an approximation of test accuracy. Here's a breakdown of the relationship:

### 1. **Difference Between Validation and Test Accuracy**
   - **Validation Accuracy**: This is the accuracy obtained from evaluating the model on a validation set, which is a subset of the training data. The model is trained on the training set and validated on the validation set. In cross-validation, this process is repeated multiple times with different splits of the data.
   - **Test Accuracy**: This is the accuracy obtained by evaluating the model on a completely separate **test set**, which is not used during training or validation. The test set is used only at the very end, after model training and hyperparameter tuning, to evaluate the final model's generalization performance.

   **Key Difference**: The test accuracy measures how well the model generalizes to truly unseen data, while validation accuracy helps in choosing the best model (or hyperparameters) but can still be influenced by how the data is split during the training and validation phases.

### 2. **Does Averaging Validation Accuracy Estimate Test Accuracy Well?**
   - **Averaging validation accuracy across multiple splits** (like in **k-fold cross-validation**) provides a **more consistent and robust estimate** of the model's ability to generalize, but it is still based on data that has been used for model selection and training (in the validation process).
   - **Why it’s not a direct estimate of test accuracy**:
     - The validation accuracy can be slightly optimistic because the model has been tuned on the training and validation sets, and it might be slightly overfitted to these specific splits.
     - The true test accuracy comes from a final, independent test set that the model has never seen before. Validation accuracy does not account for any unseen test data.
     
   - **Still, it’s a good approximation**: The more you average across different splits of the data (in cross-validation), the better the estimate you get of how the model will generalize. It smooths out the variability from individual splits and gives a more reliable approximation of test accuracy.

### 3. **Cross-Validation vs. Test Accuracy:**
   - **Cross-validation gives an approximation** of how well your model will perform on unseen data because it tests the model on multiple validation sets (each split is used as a validation set at some point), mimicking the test process by ensuring the model doesn't see any data it is evaluated on.
   - However, **test accuracy** is the gold standard for assessing how your final model will perform on truly unseen data. Cross-validation helps you tune the model and estimate generalization performance, but the **test set** is the final, unbiased evaluation of the model's ability to generalize.

### 4. **Factors That Influence Accuracy Estimates:**
   - **Overfitting on Validation Data**: When you perform cross-validation or split the data multiple times for validation, there’s a small risk that the model can overfit the validation sets. If you keep using validation data to tweak the model, the validation accuracy may be overly optimistic, and it won’t match the test accuracy exactly.
   - **Data Leakage**: If there’s any leakage (i.e., some information from the test set accidentally influences the training process), both the validation and test accuracy estimates could become biased and not reflect true performance on new, unseen data.
   - **Choice of Cross-Validation Method**: The type of cross-validation you use matters. **k-fold cross-validation** tends to give more reliable results, but if you use methods like **train-test splits** (especially with a single split), the validation accuracy might not be as stable.

### 5. **Best Practices for Estimating Test Accuracy:**
   - **Use Cross-Validation for Model Selection**: You can use cross-validation (averaging across splits) to evaluate and compare models or select hyperparameters. This gives a good approximation of how your model will perform on unseen data.
   - **Final Evaluation on Test Set**: Once you’ve selected the final model and tuned it using cross-validation, you should evaluate it **on a separate test set** that the model has never seen before. This provides the true **test accuracy**, which is the best estimate of how the model will perform in real-world scenarios.
   
   **In summary**: Cross-validation (averaging across multiple validation splits) gives a **good approximation** of test accuracy but is not an exact estimate of how the model will perform on a final, independent test set. The most accurate estimate of test accuracy comes from evaluating the model on a **separate, untouched test set** after training and hyperparameter selection.



3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

Yes, **increasing the number of iterations** (or **cross-validation folds** or **epochs**, depending on the context) can **improve the estimate** of model performance, but the effect varies depending on the specific context. Let’s break down how the number of iterations impacts your estimate in different scenarios:

### 1. **Effect of Increasing the Number of Cross-Validation Folds (Iterations)**
In the context of **k-fold cross-validation**, increasing the number of folds (iterations) has several effects:

- **More Data for Training**: In k-fold cross-validation, the data is split into **k** subsets (folds), and for each fold, the model is trained on **k-1** subsets and tested on the remaining fold. As you increase **k** (for example, from 5-fold to 10-fold), each training iteration will use a larger portion of the data (since you're testing on a smaller fold), which can lead to a **better-trained model**.
  
- **Improved Estimate of Generalization**: With more folds, each individual validation set gets to be used as a test set, providing a more **representative estimate** of how the model performs across different subsets of the data. By averaging the performance across more splits, the estimate becomes **less sensitive** to any single, potentially unrepresentative fold, and is generally more **robust**.

- **Reduced Variance**: A larger number of folds (e.g., 10-fold vs. 5-fold) helps reduce the variance in your performance estimates. This means that you get a **more stable and reliable** estimate of how the model will generalize to unseen data. Essentially, with more folds, you're averaging over more different sets of data, which smooths out random fluctuations in performance.

- **Diminishing Returns**: While increasing the number of folds generally improves the estimate, after a certain point, the **improvement becomes marginal**. For instance, moving from 5-fold to 10-fold cross-validation might show a slight improvement in consistency, but going from 100-fold to 200-fold might not substantially change the estimate. This is because with each additional fold, you're just using smaller and smaller portions of the data for testing, and the additional improvements become less noticeable.

- **Computational Cost**: Increasing the number of iterations (folds) increases the **computational cost**. More folds mean more training and testing cycles, which can be very expensive, especially with large datasets and complex models.

### 2. **Effect of Increasing the Number of Training Iterations (Epochs)**
If we are talking about **training iterations (epochs)** in a neural network or other iterative models (e.g., gradient descent), increasing the number of iterations can also affect the estimate, but in a different way:

- **Overfitting Risk**: As the number of training epochs increases, the model is exposed to the training data more times, which can allow it to better **fit the data**. However, this also increases the risk of **overfitting** to the training data, meaning the model may memorize the data rather than learning generalizable patterns. This can hurt the model’s ability to generalize to unseen data, especially if you don't have sufficient regularization or early stopping mechanisms in place.

- **Improved Convergence**: For many machine learning algorithms, especially those that use gradient-based optimization (like neural networks), increasing the number of epochs allows the model to **converge** more completely toward an optimal solution. In this case, a higher number of iterations can result in better performance **within the training set**. However, the performance improvement typically plateaus after a certain number of epochs, and too many epochs may lead to diminishing returns or overfitting.

- **Better Generalization (If Stopped Early)**: **Early stopping** is often used to avoid overfitting. The idea is to stop training after a certain number of epochs when the model starts to show signs of overfitting (e.g., when validation performance starts to degrade). In such cases, a higher number of epochs may help **find the optimal point** before overfitting becomes a problem, leading to **better generalization**.

### 3. **Effect on Estimate Quality**
Here’s how the number of iterations (cross-validation folds or training epochs) influences the **quality of your performance estimate**:

- **More Iterations = Better Estimate (Generally)**:
  - For **cross-validation**: More folds (or iterations) usually lead to a **more stable, less biased** estimate of model performance.
  - For **training epochs**: More iterations can help the model **converge to a better solution**, but you need to balance this with the risk of overfitting. It's not just about the number of epochs; it's about the **training-validation performance relationship**.

- **Diminishing Returns**: After a certain point, you may see **diminishing returns** with respect to the improvement in your estimate. For example, going from 5-fold to 10-fold cross-validation usually gives a better estimate, but increasing to 20-fold doesn’t offer a significantly better result unless you have a very large dataset.

- **Computational Efficiency**: More iterations take more computational resources, so it's important to find a balance between accuracy and computational efficiency. You want to improve the stability of your estimate without making the process unnecessarily long or expensive.

### 4. **Trade-off Between Quality and Cost**
Increasing iterations can lead to a **better estimate**, but it also comes with a trade-off:

- **Cross-validation**: More folds mean more time spent training the model. For smaller datasets, increasing folds might provide significant improvement in the estimate, but for very large datasets, the benefits might be minimal compared to the computational cost.
- **Training iterations**: More training epochs can help the model converge more thoroughly but may increase the risk of overfitting or just be inefficient if the model is already converging well. It is important to have proper early stopping mechanisms to prevent wasting resources.

### Conclusion
- **Yes**, in general, **higher iterations (cross-validation folds or training epochs) provide a more accurate and reliable estimate** of model performance, but only up to a point. The performance improves by averaging over more data splits or allowing more time for training, but after a certain point, the improvements become marginal and the **computational cost** increases.
- **Cross-validation**: Increasing the number of folds typically gives a more **stable estimate** of model performance.
- **Training epochs**: More iterations (epochs) can improve model training, but careful monitoring for overfitting and employing techniques like **early stopping** is necessary.

Thus, higher iterations usually result in a better estimate, but **balance** is crucial in terms of computational cost and the risk of overfitting.


4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Here's an answer to guide you:

No, simply increasing the number of iterations is unlikely to solve the issue of a very small training or validation dataset. While more iterations might allow the model to better fit the limited data it has, it could lead to overfitting, where the model learns noise and specific details of the small dataset instead of generalizable patterns. This reduces the model's ability to perform well on unseen data.

A more effective approach could involve:

1. Data Augmentation: Generating new data from the small dataset by applying transformations (e.g., rotations, flips, or noise for images).


2. Transfer Learning: Using a pre-trained model and fine-tuning it on the small dataset.


3. Regularization: Adding techniques like dropout or weight decay to prevent overfitting.

4. Synthetic Data: Creating artificial data using methods like GANs or statistical techniques.


5. Cross-validation: Making the most out of the limited data by using it iteratively for training and validation.



> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.